In [2]:
import pandas as pd
import numpy as np

In [3]:
url = '/Users/vigneshpadmanabhan/Downloads/vehicles.csv.zip'
df = pd.read_csv(url, low_memory=False)
city_mpg = df.city08
highway_mpg = df.highway08